In [ ]:
from datetime import datetime, timedelta

from metpy.plots import declarative
from metpy.units import units
from siphon.catalog import TDSCatalog
import xarray as xr

In [ ]:
# Set the date/time of the model run
### SET THE APPROPRIATE DATE/TIME ###
date = datetime(2019, 3, 14, 0)
#####################################

# GFS 0.5 degree Analysis back to 06 UTC 24 December 2007 to five days prior to present
ds = xr.open_dataset('https://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files-old/'
                     f'{date:%Y%m/%Y%m%d}/gfsanl_4_{date:%Y%m%d_%H}00_000.grb2').metpy.parse_cf()

# Subset data to be just over the U.S. for plotting purposes
ds = ds.sel(lat=slice(70,10), lon=slice(360-150, 360-55))

In [ ]:
hght_500 = ds.Geopotential_height_isobaric.metpy.sel(time=date, vertical=500 * units.hPa).metpy.quantify()
hght_1000 = ds.Geopotential_height_isobaric.metpy.sel(time=date, vertical=1000 * units.hPa).metpy.quantify()
ds['thickness'] = (hght_500 - hght_1000).metpy.dequantify()

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

# Set attributes for plotting contours
cntr = declarative.ContourPlot()
cntr.data = ds
cntr.field = 'Pressure_reduced_to_MSL_msl'
cntr.level = None
cntr.time = plot_time

### Set the appropriate contour interval for MSLP ###
cntr.contours = list(range(0, 100000, 1200))
#####################################################

cntr.clabels = True

### Set the appropriate units for MSLP ###
cntr.plot_units = None
##########################################

# Set attributes for plotting Temperature Contours
cfill = declarative.FilledContourPlot()
cfill.data = ds
cfill.field = 'thickness'
cfill.level = None
cfill.time = None
cfill.contours = list(range(5400-600, 5400+600, 6))
cfill.colormap = 'coolwarm'
cfill.colorbar = 'horizontal'

# Add wind barbds
barbs = declarative.BarbPlot()
barbs.data = ds
barbs.time = plot_time
barbs.field = ['u-component_of_wind_height_above_ground',
               'v-component_of_wind_height_above_ground']
barbs.level = 10 * units.meter

### Reduce the number of wind barbs to an appropriate level ###
barbs.skip = (1, 1)
#######################################################

barbs.plot_units = 'knot'

# Set the attributes for the map
# and put the contours on the map
panel = declarative.MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']

#### ADD A TITLE ###
panel.title = f''
####################

panel.plots = [cfill, cntr, barbs]

# Set the attributes for the panel
# and put the panel in the figure
pc = declarative.PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()

### Uncomment line below and SAVE FIGURE - Change figure name ###
#pc.save(f'test.png', dpi=150, bbox_inches='tight')
########################################